In [1]:
#La probabilidad de que la versión actual de un contenido fuera editada sin dejar comentario para usuarios que están logueados y que no están logueados (⭐)

In [2]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk+
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=2880bdd1e6fba82613bc0de346400910eb5b854bd3807e0ed1536bafb57ca562
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark
Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ Packages [76.4 kB]
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:7 https:/

In [3]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [13]:
sqlContext = SQLContext(sc)
df = sqlContext.read.csv('/content/drive/MyDrive/Organizacion De Datos/Trabajo_practico/contents.csv', header=True, inferSchema=True)
rdd = df.rdd

/usr/local/lib/python3.9/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Me quedo con las revisiones por usuarios registrados

In [21]:
rdd_users_registrados = rdd.filter(lambda x: x.revisor_id).cache()

In [42]:
tot_revisiones_usuarios_registrados = rdd_users_registrados.count()

Me quedo con las revisiones por usuarios no registrados

In [23]:
rdd_users_no_registrados = rdd.filter(lambda x: not x.revisor_id).cache()

In [38]:
tot_revisiones_usuarios_no_registrados = rdd_users_no_registrados.count()

Dentro de los usuarios registrados, me quedo con aquellas revisiones sin comentario

In [27]:
revisiones_usuarios_registrados_sin_coment = rdd_users_registrados.filter(lambda x: not x.revisor_comment).count()

Dentro de los usuarios NO registrados, me quedo con aquellas revisiones sin comentario

In [28]:
revisiones_usuarios_no_registrados_sin_coment = rdd_users_no_registrados.filter(lambda x: not x.revisor_comment).count()

Probabilidad pedida para usuarios registrados

In [43]:
print("{} %".format(revisiones_usuarios_registrados_sin_coment*100/tot_revisiones_usuarios_registrados))

9.957152762068986 %


Probabilidad pedida para usuarios NO registrados

In [41]:
print("{} %".format((revisiones_usuarios_no_registrados_sin_coment*100)/tot_revisiones_usuarios_no_registrados) )

40.021470302855136%
